In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/training"

#### Interpolating Data across patient timeseries

In [4]:
finalMerge = pd.read_csv('../eICU' + database_type + '/finalMerge.csv')
finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1648633,141764,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.003003,NaN
65,141764,4.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,141764,9.0,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,141764,14.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,141764,19.0,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,NaN,87.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634633,3353113,2676.0,NaN,85.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634665,3353113,2681.0,NaN,84.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
1634621,3353113,2686.0,NaN,84.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
finalMerge.loc[(finalMerge['patientunitstayid'] == 346380)].sort_values(by='observationoffset')

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1647664,346380,34.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1677078,346380,194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677077,346380,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677074,346380,685.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677076,346380,2255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677079,346380,2310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677073,346380,5028.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1677075,346380,5035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [6]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))
finalData = pd.DataFrame()

In [7]:
for id in idList[:51]:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    sepsisDiagnosisTime = df[df['diagnosis'] == 1].observationoffset
    if len(sepsisDiagnosisTime) != 0:
        sepsisDiagnosisTimeDict = sepsisDiagnosisTime.to_dict()
        sepsisDiagnosisTime = sepsisDiagnosisTimeDict[list(sepsisDiagnosisTimeDict.keys())[0]]
        df = df[df['observationoffset'] <= sepsisDiagnosisTime]
    df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
    df.loc[:, 'diagnosis'] = df['diagnosis'].interpolate(method='linear')
    df['diagnosis'].fillna('0', inplace=True)
    finalData = finalData.append(df)

finalData

C:\tools\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\tools\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1648633,141764,-14.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
65,141764,4.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
12,141764,9.0,NaN,104.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
63,141764,14.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
59,141764,19.0,NaN,98.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
...,...,...,...,...,...,...,...,...,...,...,...
16553,167186,196.0,NaN,88.0,25.0,138.0,1.29,NaN,NaN,5.300353,0
16571,167186,201.0,NaN,86.0,26.0,138.0,1.29,NaN,NaN,5.300353,0
16543,167186,206.0,NaN,86.0,28.0,138.0,1.29,NaN,NaN,5.300353,0
16551,167186,211.0,NaN,88.0,28.0,138.0,1.29,NaN,NaN,5.300353,0


In [8]:
finalData

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1648633,141764,-14.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
65,141764,4.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
12,141764,9.0,NaN,104.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
63,141764,14.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
59,141764,19.0,NaN,98.0,NaN,NaN,NaN,NaN,NaN,3.003003,0
...,...,...,...,...,...,...,...,...,...,...,...
16553,167186,196.0,NaN,88.0,25.0,138.0,1.29,NaN,NaN,5.300353,0
16571,167186,201.0,NaN,86.0,26.0,138.0,1.29,NaN,NaN,5.300353,0
16543,167186,206.0,NaN,86.0,28.0,138.0,1.29,NaN,NaN,5.300353,0
16551,167186,211.0,NaN,88.0,28.0,138.0,1.29,NaN,NaN,5.300353,0


In [9]:
# path = "../eICU/training/"
# finalData.to_csv(path + "finalData.csv", sep=',', index=False, encoding='utf-8')
